### Define URL for download and initiate webdriver

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()  # Or whichever browser/driver you're using


# Insert URL of the reviews page of the place
url1 = 'https://www.google.com/maps/place/Old-Phoenix+Rooms+%26+Restaurant/@35.1999187,24.0721182,15z/data=!4m11!3m10!1s0x136061919fe41577:0x609b596cdbd2b8be!5m2!4m1!1i2!8m2!3d35.1999187!4d24.0721182!9m1!1b1!16s%2Fg%2F1tml7nv7?entry=ttu'

url2 = 'https://www.google.com/maps/place/Villa+Niki/@35.1991542,24.0706977,16.41z/data=!4m24!1m12!3m11!1s0x136061919fe41577:0x609b596cdbd2b8be!2sOld-Phoenix+Rooms+%26+Restaurant!5m2!4m1!1i2!8m2!3d35.1999187!4d24.0721182!9m1!1b1!16s%2Fg%2F1tml7nv7!3m10!1s0x149c9ee60031954f:0x939da5cbfcd24dbc!5m2!4m1!1i2!8m2!3d35.200207!4d24.078342!9m1!1b1!16s%2Fg%2F12hy_j8gk?hl=en&entry=ttu'
driver.get(url2)

### Translating the webpage into English

In [31]:

# xpath = "//button[contains(., 'ביקורות') or contains(., 'REVIEWS')]"
wait = WebDriverWait(driver, 2)
menu_xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[1]/ul/li[1]/button/div'
menu_xpath = wait.until(EC.element_to_be_clickable((By.XPATH, menu_xpath)))
menu_xpath.click()
wait = WebDriverWait(driver, 1)
lan = '//*[@id="settings"]/div/div[2]/ul/div[7]/li[1]/button'
language_button = wait.until(EC.element_to_be_clickable((By.XPATH, lan)))
language_button.click()
wait = WebDriverWait(driver, 1)

# click on the english language
english_xpath = '//*[@id="modal-dialog"]/div/div[2]/div/div[2]/div/div/div/div[3]/div[1]/div[11]/a'
anchor_element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, english_xpath))
)
anchor_element.click()

In [34]:
# print the total number of reviews
xpath = "//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]/div[2]/div/div[2]/div[3]"
total_number_of_reviews = driver.find_element(By.XPATH, xpath).text.split(" ")[0]
total_number_of_reviews = int(total_number_of_reviews.replace(',','')) if ',' in total_number_of_reviews else int(total_number_of_reviews)
print(f"The place has {total_number_of_reviews} reviews on google")

The place has 83 reviews on google


### Scrolling down the webpage

In [35]:
import time
from selenium.webdriver.common.keys import Keys


# click on the "more" button on long reviews
def expand_more_buttons(driver):
    # Using the aria-label to find the "More" buttons
    more_buttons = driver.find_elements(By.XPATH, "//button[@aria-label='See more']")

    for button in more_buttons:
        try:
            # Scroll the button into view first
            driver.execute_script("arguments[0].scrollIntoView();", button)
            # Wait until the button is clickable
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable(button)).click()
        except Exception as e:
            print("Could not click on a 'More' button:", e)
            continue  # If a button can't be clicked, move to the next one

# Scrolling by sending PAGE_DOWN key presses
body = driver.find_element(By.XPATH, "//div[contains(@class, 'm6QErb') and contains(@class, 'DxyBCb') and contains(@class, 'kA9KIf') and contains(@class, 'dS8AEf')]")
for _ in range(total_number_of_reviews):  # Adjust the range according to the number of scrolls needed
    body.send_keys(Keys.PAGE_DOWN)
    expand_more_buttons(driver)
    time.sleep(1)

Could not click on a 'More' button: Message: element click intercepted: Element <button class="w8nwRe kyuRq" aria-expanded="false" aria-controls="ChdDSUhNMG9nS0VJQ0FnSUQwdVpqUm9RRRAB" data-review-id="ChdDSUhNMG9nS0VJQ0FnSUQwdVpqUm9RRRAB" jslog="63707; track:click;metadata:WyIwYWhVS0V3aVBfNWlYdmN1RkF4WHp3Z0lISGZUOURKMFEwcE1GQ0FJb0FBIl0=" aria-label="See more" jsaction="pane.wfvdle40.review.expandReview">...</button> is not clickable at point (210, 222). Other element would receive the click: <span class="xRkPPb">...</span>
  (Session info: chrome=123.0.6312.107)
Stacktrace:
	GetHandleVerifier [0x00007FF720DE7032+63090]
	(No symbol) [0x00007FF720D52C82]
	(No symbol) [0x00007FF720BEEC65]
	(No symbol) [0x00007FF720C3BB34]
	(No symbol) [0x00007FF720C39954]
	(No symbol) [0x00007FF720C37164]
	(No symbol) [0x00007FF720C35EF9]
	(No symbol) [0x00007FF720C2A708]
	(No symbol) [0x00007FF720C56FDA]
	(No symbol) [0x00007FF720C2A00A]
	(No symbol) [0x00007FF720C571F0]
	(No symbol) [0x00007FF720C73412]


KeyboardInterrupt: 

### Save reviews into a dataframe

In [36]:
from bs4 import BeautifulSoup
import pandas as pd


# reviews = driver.find_elements(By.XPATH, "//div[@data-review-id]")

# define an empty DataFrame for all of the reviews
df = pd.DataFrame(columns=['text_review', 'stars', 'time'])

# Assuming `driver.page_source` contains the HTML source of the page with reviews
html_content = driver.page_source
response = BeautifulSoup(html_content, 'html.parser')
reviews = response.find_all('div', class_='jJc9Ad')

review_data = []
for review in reviews:
    review_text = "No review text found"
    star_rating = "No rating found"

    review_text_div = review.find('div', class_='MyEned')
    if review_text_div:
        review_text_span = review_text_div.find('span', class_='wiI7pd')
        if review_text_span:
            review_text = review_text_span.text

    stars_span = review.find('span', class_='fzvQIb')
    if stars_span:
        stars = stars_span.text[0]
        
    time_span = review.find('span', class_='xRkPPb')
    if time_span:
        time = time_span.text

    review_data.append({'text': review_text, 'stars': stars, 'time':time})

# Create a DataFrame from the review_data list
df_reviews = pd.DataFrame(review_data)

# remove empty reviews
df_reviews = df_reviews[df_reviews['text'] != "No review text found"]


In [37]:
print(f"Number of reviews collected: {len(df_reviews)}")
# Display the DataFrame to verify its content
df_reviews

Number of reviews collected: 28


,text,stars,time
0,Excellent stay! Fantastic view from the verand...,5,5 months ago on Tripadvisor
1,Owner was available and accommodating during e...,5,6 months ago on Google
2,Just steps from the beach Villa Niki was spaci...,5,a year ago on Tripadvisor
3,Andreas and his wife are amazing warm and frie...,5,a year ago on Tripadvisor
4,It couldn't be more welcoming in a guest house...,5,2 years ago on Tripadvisor
5,"This is a very clean and friendly ""Rooms to le...",5,2 years ago on Tripadvisor
6,"Great, family run hotel full of character, a l...",5,4 years ago on Tripadvisor
7,"Like all the other reviews mention, Andreas an...",5,4 years ago on Google
8,Excellent! There are two main reasons - I beli...,5,4 years ago on Google
9,We spent 1 night here: we had a super nice ro...,5,4 years ago on Tripadvisor


In [38]:
import numpy as np
import re

# Function to convert time strings to months
def time_to_months(time_str):
    # Extract the number and unit (year/month)
    match = re.search(r'(\d+|\ba\b)\s+(year|month)', time_str)
    if match:
        number_str, unit = match.groups()
        number = 1 if number_str == 'a' else int(number_str)
        
        if 'year' in unit:
            return number * 12  # Convert years to months
        elif 'month' in unit:
            return number
    return np.nan  # Return NaN for unmatched strings

# Apply the function to the 'time' column
df_reviews['months_ago'] = df_reviews['time'].apply(time_to_months)

df_reviews.drop('time', axis=1, inplace=True)
df_reviews

,text,stars,months_ago
0,Excellent stay! Fantastic view from the verand...,5,5
1,Owner was available and accommodating during e...,5,6
2,Just steps from the beach Villa Niki was spaci...,5,12
3,Andreas and his wife are amazing warm and frie...,5,12
4,It couldn't be more welcoming in a guest house...,5,24
5,"This is a very clean and friendly ""Rooms to le...",5,24
6,"Great, family run hotel full of character, a l...",5,48
7,"Like all the other reviews mention, Andreas an...",5,48
8,Excellent! There are two main reasons - I beli...,5,48
9,We spent 1 night here: we had a super nice ro...,5,48


In [39]:
df_reviews.to_csv('reviews_Villa Niki.csv', index=False)